In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.svm import LinearSVC

In [3]:
def dec2bin(i):
    X = [0]*4;
    X[0] = i // 8;
    i = i-(8*X[0])
    X[1] = i // 4;
    i = i-(4*X[1])
    X[2] = i // 2;
    i = i-(2*X[2]);
    X[3] = i;
    return X

In [4]:
def bin2dec(X):
    return 8*X[0]+4*X[1]+2*X[2]+X[3]

In [5]:
def swap(X, Y):
    X[0], Y[0]  = Y[0], X[0]
    X[1], Y[1]  = Y[1], X[1]
    X[2], Y[2]  = Y[2], X[2]
    X[3], Y[3]  = Y[3], X[3]

In [6]:
train_data = np.loadtxt( "train.dat" )
train_data.shape


test_data = np.loadtxt( "test.dat" )
test_data.shape

(40000, 73)

In [7]:
def preprocess(X):
    for row in X:
        if(int(bin2dec(row[64:68])) >= int(bin2dec(row[68:72]))):
            swap(row[64:68], row[68:72])
            row[72] = 1 - row[72]
    return X

train_data = preprocess(train_data)
data = train_data

test_data = preprocess(test_data)

In [19]:
models = [[None]*16]*16

counter = 0
for i in range(16):
    X = dec2bin(i);
    for j in range(i+1,16):
        Y = dec2bin(j)
        mask1 = np.where((data[:,64]==X[0]) & (data[:,65]==X[1]) & (data[:,66]==X[2]) & (data[:,67]==X[3]) & (data[:,68]==Y[0]) & (data[:,69]==Y[1]) & (data[:,70]==Y[2]) & (data[:,71]==Y[3]))
        a = data[mask1[0]]
        data_trn, data_tst = train_test_split(a, train_size = 0.85)

        # Insufficient training data can lead to low accuracy
        # Try using train_size = 30000 or 3000 or 300 or 30

        clf = LinearSVC( loss = "squared_hinge", max_iter=1800)
        clf.fit(data_trn[:,:-9], data_trn[:,-1] )
        counter += (np.average( data_tst[:,-1] == clf.predict(data_tst[:,:-9] ) ) )
        
        models[i][j] = clf

C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\UDVAS BASAK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [21]:
for i in range(16):
    for j in range(i+1,16):
        print(models[i][j].coef_)

[[ 0.51641213 -0.22362043  1.05879692 -0.21784506 -1.51629716 -0.08958573
  -0.28628217 -0.4099332   0.49713795 -0.56320863 -0.85444673  0.08505501
   0.54114795  0.65168114 -0.58845069 -0.59629592 -0.41599649 -0.01504073
   0.16988292  0.33016075  0.68880406  0.05199982 -0.51928481 -0.2926888
  -0.53519802  0.7784401   1.14105254  0.39737459 -0.04091977  0.81106377
   0.84699676  0.7467242   0.3386782   0.91371651  0.01305925 -1.11468123
  -0.96580923 -0.93648682 -0.32588462 -0.07478863  0.12085399  0.81194716
   0.04843288 -0.5444498   0.38856879  0.69355938  1.254459    0.9631973
   0.22561115  0.26388707 -0.83402059  1.11308553 -0.97375192 -0.60917908
  -0.10650629  1.1429972  -0.65821793 -0.01580346 -0.3618824   0.9293586
  -0.8194549   0.80833997 -0.19288508  0.76275164]]
[[-0.69429399 -0.47851607 -0.18195678 -0.16873476  0.68042002  0.89164475
   0.64463315  0.79033018  0.0310952   0.18245409  1.75538864 -0.34570771
  -0.80980594 -0.79720776  1.40830333  0.39291174  1.70676332  

In [14]:
a = 0

print(models[3][4].coef_.shape)

for row in train_data:
    x = int(bin2dec(row[64:68]))
    y = int(bin2dec(row[68:72]))
    clf = models[x][y]
    a += (row[-1] == clf.predict(row[:-9].reshape(1,-1) ) ) 

(1, 64)
